### CellStrat Hub Pack - Deep Learning

#### DL10 - Building LSTM model for text data and getting the results

### Start by importing the SimpleRNN layer

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN
import tensorflow as tf
print('tf version', tf.__version__)

model = Sequential()

# Word embeddings are dense representation of words and their relative meanings. 
# They can be learned from text data and reused among projects. 
# They can also be learned as part of fitting a neural network on text data.

model.add(Embedding(10000, 32))
model.add(SimpleRNN(32))

tf version 2.5.0


### Let's see how the model looks

It has over 322,000 parameters

In [2]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          320000    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 32)                2080      
Total params: 322,080
Trainable params: 322,080
Non-trainable params: 0
_________________________________________________________________


In [3]:
model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(32, return_sequences=True))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, None, 32)          2080      
Total params: 322,080
Trainable params: 322,080
Non-trainable params: 0
_________________________________________________________________


It is sometimes useful to stack several recurrent layers one after the other in order to increase the representational power of a network. 

In [4]:
model = Sequential()
model.add(Embedding(10000, 32))

model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32, return_sequences=True))
# return_sequences: Boolean. Whether to return the last output
# in the output sequence, or the full sequence.
model.add(SimpleRNN(32))

In [5]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, None, 32)          2080      
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, None, 32)          2080      
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, None, 32)          2080      
_________________________________________________________________
simple_rnn_5 (SimpleRNN)     (None, 32)                2080      
Total params: 328,320
Trainable params: 328,320
Non-trainable params: 0
_________________________________________________________________


Now let's try to use such a model on the IMDB movie review classification problem. First, let's preprocess the data:

In [6]:
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 10000  # number of words to consider as features
maxlen = 500  # cut texts after 500 words
batch_size = 32

print('Loading data...')
(input_train, y_train), (input_test, y_test) = imdb.load_data(num_words=max_features)
print(len(input_train), 'train sequences')
print(len(input_test), 'test sequences')

print('Pad sequences (samples x time)')
input_train = sequence.pad_sequences(input_train, maxlen=maxlen)
input_test = sequence.pad_sequences(input_test, maxlen=maxlen)
print('input_train shape:', input_train.shape)
print('input_test shape:', input_test.shape)

Loading data...
17473536/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this,

25000 train sequences
25000 test sequences
Pad sequences (samples x time)
input_train shape: (25000, 500)
input_test shape: (25000, 500)


Let's train a simple recurrent network using an `Embedding` layer and a `SimpleRNN` layer:

In [7]:
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

history = model.fit(input_train, y_train,
                    epochs=1,
                    batch_size=128,
                    validation_split=0.2)

157/157 [==============================] - 25s 151ms/step - loss: 0.6569 - acc: 0.6036 - val_loss: 0.6123 - val_acc: 0.6682


Let's display the training and validation loss and accuracy:

In [8]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']


print('Training set accuracy is: ', acc)
print('Validation set accuracy is: ', val_acc)
print('Training set Loss is: ', loss)
print('Validation set accuracy is: ', val_loss)

# Of course, you can train it for larger epochs
# to improve the accuracy

Training set accuracy is:  [0.6035500168800354]
Validation set accuracy is:  [0.6682000160217285]
Training set Loss is:  [0.6568548083305359]
Validation set accuracy is:  [0.6123234033584595]


## 2) Same Example with LSTM - Long Short-term Memory Layer

In [9]:
from tensorflow.keras.layers import LSTM

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(input_train, y_train,
                    epochs=1,
                    batch_size=128,
                    validation_split=0.2)

157/157 [==============================] - 49s 301ms/step - loss: 0.4875 - acc: 0.7750 - val_loss: 0.3148 - val_acc: 0.8750


In [10]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']


print('Training set accuracy is: ', acc)
print('Validation set accuracy is: ', val_acc)
print('Training set Loss is: ', loss)
print('Validation set accuracy is: ', val_loss)

# You can train it for larger epochs to improve the accuracy

Training set accuracy is:  [0.7749500274658203]
Validation set accuracy is:  [0.875]
Training set Loss is:  [0.4874570667743683]
Validation set accuracy is:  [0.3147842586040497]
